In [19]:
import os
from langgraph.graph import StateGraph,START,END,MessagesState
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage,ToolMessage,AIMessage      
from langchain_openai import AzureChatOpenAI,AzureOpenAIEmbeddings
from dotenv import load_dotenv,find_dotenv
from IPython.display import Image

load_dotenv(find_dotenv(),override=True)
endpoint = os.getenv("ENDPOINT_URL")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version="2025-01-01-preview"

llm = AzureChatOpenAI(azure_endpoint=endpoint,api_key=subscription_key,api_version=api_version)

@tool
def currentTemperature(country:str) -> int :
    '''current temperature of the country'''
    if country.startswith("i"):
         return 10
    elif country.startswith("u"):
        return 20
    else:
        return 30

tool_mapping={
    "currentTemperature":currentTemperature
}
tools=[currentTemperature]

def call_model(state:MessagesState):
    messages=state["messages"]
    llm_tool=llm.bind_tools(tools)
    result=llm_tool.invoke(messages)
    return {"messages": [result]}

def should_continue(state:MessagesState):
    messages=state["messages"]
    last_message=messages[-1]
    if last_message.tool_calls:
        return "tools"
    else:
        return END

graph=StateGraph(MessagesState)
tool_node=ToolNode(tools)

graph.add_node("agent",call_model)
graph.add_node("tools",tool_node)

graph.add_edge(START,"agent")
graph.add_conditional_edges("agent",should_continue)
graph.add_edge("tools","agent")

runnable=graph.compile()
# Image(runnable.get_graph().draw_mermaid_png())

res=runnable.invoke({"messages":[HumanMessage(content="what is the current temperature in china?")]})
res

{'messages': [HumanMessage(content='what is the current temperature in china?', additional_kwargs={}, response_metadata={}, id='1acfd987-66e6-42a6-b740-71e5f35796f7'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_OujTPXB7feENV4NotmEbUSQ7', 'function': {'arguments': '{"country":"China"}', 'name': 'currentTemperature'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 51, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_7a7fd0eb44', 'id': 'chatcmpl-DBS9nGk0X8WUsty5UMNbbNbWX973U', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_har